# GNN-Based Relation Extraction with Contrastive Learning
This Jupyter notebook presents a comprehensive approach to extracting relations from TACRED text data using Graph Neural Networks (GNN) augmented by contrastive learning. The methodology is structured into several key sections resulting in a model capable of identifying and classifying relations within sentences.

### 1. Setting Up and Utilizing Key Libraries for Graph Neural Networks

This set of code cells prepares the environment for graph neural networks. It includes the installation of necessary Python libraries, imports of various packages for data handling, numerical operations, and evaluation metrics. The setup is essential for performing tasks related to deep learning, graph data processing, and performance evaluation.

In [ ]:
# This line installs necessary Python packages for the code to run
!pip install numpy torch torch-geometric scikit-learn

In [ ]:
# Importing necessary Python packages
import json  # For handling JSON data
import random  # For generating random numbers
import numpy as np  # For numerical operations
import torch  # PyTorch library for deep learning
import torch.nn.functional as F  # Functional interface for common operations
from torch.nn import CrossEntropyLoss  # Loss function for classification tasks
from torch_geometric.data import Data, DataLoader  # Handling graph data and DataLoader for batching
from torch_geometric.nn import GATConv, global_mean_pool  # Graph Attention Network layers
from sklearn.metrics import accuracy_score, precision_recall_fscore_support  # Evaluation metrics

### 2. Data Preprocessing and Stratified Sampling

This section focuses on two main tasks: reading data from JSON files and creating stratified subsets of data. Initially, we define a function read_json_file to load JSON data into dictionaries. This function is utilized to read training, development, and test datasets. We introduce the select_stratified_subset function, which selects a stratified subset of the data, ensuring an even distribution based on the 'relation' attribute. This ensures our subsets are representative of the original datasets.

In [ ]:
def read_json_file(file_path):
    """
    Function to read JSON data from a file.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        dict: JSON data read from the file.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Reading train, dev, and test data from respective JSON files
train_data = read_json_file('train.json')
dev_data = read_json_file('dev.json')
test_data = read_json_file('test.json')

In [ ]:
def select_stratified_subset(data, subset_size):
    """
    Function to select a stratified subset of data based on the distribution of relations.

    Args:
        data (list): List of dictionaries representing data items, each containing a 'relation' key.
        subset_size (int): Size of the desired subset.

    Returns:
        list: Stratified subset of data.
    """
    # Initialize a dictionary to count the occurrences of each relation
    relation_counts = {}

    # Count occurrences of each relation in the data
    for item in data:
        relation = item['relation']
        if relation not in relation_counts:
            relation_counts[relation] = []
        relation_counts[relation].append(item)

    # Calculate the total number of items across all relations
    total_items = sum(len(items) for items in relation_counts.values())

    # Calculate the desired number of items to sample for each relation
    subset_counts = {relation: int(len(items) / total_items * subset_size) for relation, items in relation_counts.items()}

    # Ensure at least one sample per relation if subset size allows
    for relation in subset_counts:
        if subset_counts[relation] == 0 and subset_size > 0:
            subset_counts[relation] = 1
            subset_size -= 1  # Adjust subset_size for added samples

    # Sample items from each relation to form the subset
    subset = []
    for relation, items in relation_counts.items():
        if subset_counts[relation] > 0:
            sampled_items = random.sample(items, subset_counts[relation])
            subset.extend(sampled_items)

    return subset

# Define the desired subset sizes for train, dev, and test datasets
train_subset_size = 10000
dev_subset_size = 500
test_subset_size = 500

# Generate stratified subsets for train and dev datasets
stratified_train_data = select_stratified_subset(train_data, train_subset_size)
stratified_dev_data = select_stratified_subset(dev_data, dev_subset_size)

### 3. Preprocessing for GNN-Based Relation Extraction

This section prepares data for Graph Neural Network (GNN) based relation extraction with a focus on contrastive learning. It includes steps to mark entity tokens within sentences, create mappings for part-of-speech (POS) and named entity recognition (NER) tags to indices, one-hot encode these tags, and finally preprocess datasets by integrating these features.

In [ ]:
# Function to add entity markers to tokens
def mark_entities(tokens, subj_start, subj_end, obj_start, obj_end):
    """
    Add entity markers to tokens to indicate subject and object entities.

    Args:
        tokens (list): List of tokens representing the sentence.
        subj_start (int): Start index of the subject entity.
        subj_end (int): End index of the subject entity.
        obj_start (int): Start index of the object entity.
        obj_end (int): End index of the object entity.

    Returns:
        list: Tokens with entity markers added.
    """
    marked_tokens = []
    for idx, token in enumerate(tokens):
        if idx == subj_start:
            marked_tokens.append("<subj>")
        if idx == obj_start:
            marked_tokens.append("<obj>")
        marked_tokens.append(token)
        if idx == subj_end:
            marked_tokens.append("</subj>")
        if idx == obj_end:
            marked_tokens.append("</obj>")
    return marked_tokens

def create_tag_indices(data):
    """
    Create tag indices for part-of-speech (POS) and named entity recognition (NER) tags.

    Args:
        data (list): List of data items containing POS and NER tags.

    Returns:
        dict: Mapping from POS tags to indices.
        dict: Mapping from NER tags to indices.
    """
    pos_tags = set()
    ner_tags = set()
    for item in data:
        pos_tags.update(item['stanford_pos'])
        ner_tags.update(item['stanford_ner'])
    pos_tag_to_index = {tag: idx for idx, tag in enumerate(pos_tags)}
    ner_tag_to_index = {tag: idx for idx, tag in enumerate(ner_tags)}
    return pos_tag_to_index, ner_tag_to_index

def one_hot_encode_tags(tags, tag_to_index):
    """
    One-hot encode tags based on their indices.

    Args:
        tags (list): List of tags to encode.
        tag_to_index (dict): Mapping from tags to indices.

    Returns:
        numpy.ndarray: One-hot encoded representation of the tags.
    """
    encoded_tags = np.zeros(len(tag_to_index))
    for tag in tags:
        index = tag_to_index[tag]
        encoded_tags[index] = 1
    return encoded_tags

def preprocess_dataset(dataset, pos_tag_to_index, ner_tag_to_index):
    """
    Preprocess dataset by adding entity markers to tokens and one-hot encoding POS and NER tags.

    Args:
        dataset (list): List of data items to preprocess.
        pos_tag_to_index (dict): Mapping from POS tags to indices.
        ner_tag_to_index (dict): Mapping from NER tags to indices.

    Returns:
        list: Preprocessed dataset.
    """
    processed_dataset = []
    for item in dataset:
        processed_item = item.copy()
        processed_item['tokens'] = mark_entities(
            item['token'],
            item['subj_start'], item['subj_end'],
            item['obj_start'], item['obj_end']
        )
        processed_item['stanford_pos_one_hot'] = [one_hot_encode_tags([tag], pos_tag_to_index) for tag in item['stanford_pos']]
        processed_item['stanford_ner_one_hot'] = [one_hot_encode_tags([tag], ner_tag_to_index) for tag in item['stanford_ner']]
        processed_dataset.append(processed_item)
    return processed_dataset

# Create tag indices for the stratified train data
pos_tag_to_index, ner_tag_to_index = create_tag_indices(stratified_train_data)

# Preprocess the stratified train and dev data
preprocessed_train_data = preprocess_dataset(stratified_train_data, pos_tag_to_index, ner_tag_to_index)
preprocessed_dev_data = preprocess_dataset(stratified_dev_data, pos_tag_to_index, ner_tag_to_index)

### 4. Encoding Relations and Creating Graph Data

In this segment, we organize training data relation types into a structured format and convert relations to one-hot encodings for use in GNN models. Additionally, we introduce a method for computing the shortest dependency path between entities, an essential step for understanding the structural relationships in sentences. This helps in the creation of graph data from our preprocessed dataset. These graphs, which encapsulate entities, their relations, and their shortest dependency paths, are crucial for training our GNN-based relation extraction model with contrastive learning techniques.

In [ ]:
# Create a sorted list of unique relation types from the training data
relation_types = sorted(list(set([item['relation'] for item in preprocessed_train_data])))

# Create a mapping from each relation to a unique index based on the training data
relation_to_index = {relation: idx for idx, relation in enumerate(relation_types)}

In [ ]:
def relation_to_one_hot(relation):
    """
    Convert relation to one-hot encoding.

    Args:
        relation (str): Relation label.

    Returns:
        torch.Tensor: One-hot encoded tensor representing the relation.
    """
    # Initialize one-hot tensor with zeros
    one_hot = torch.zeros(len(relation_types), dtype=torch.float)
    # Find the index of the relation and set the corresponding position to 1
    index = relation_to_index[relation]
    one_hot[index] = 1
    return one_hot

def get_shortest_dependency_path(dependency_heads, subj_start, obj_start):
    """
    Find the shortest dependency path between subject and object entities.

    Args:
        dependency_heads (list): List of dependency heads for each token.
        subj_start (int): Start index of the subject entity.
        obj_start (int): Start index of the object entity.

    Returns:
        list: Indices representing the shortest dependency path between the entities.
    """
    # Construct a token graph based on dependency heads
    token_graph = {i: [] for i in range(-1, len(dependency_heads))}
    for i, head in enumerate(dependency_heads):
        adjusted_head = head - 1
        if adjusted_head in token_graph:
            token_graph[adjusted_head].append(i)
            token_graph[i].append(adjusted_head)
    # BFS to find the shortest path
    queue = [(subj_start, [subj_start])]
    visited = set()
    while queue:
        current, path = queue.pop(0)
        if current == obj_start:
            return path
        if current in visited:
            continue
        visited.add(current)
        for neighbor in token_graph[current]:
            if neighbor not in visited and neighbor != -1:
                queue.append((neighbor, path + [neighbor]))
    return []

def create_graphs(dataset):
    """
    Create graph data from the dataset.

    Args:
        dataset (list): List of preprocessed data items.

    Returns:
        list: List of graph data objects.
    """
    graphs = []
    for item in dataset:
        entity_to_id = {}
        node_features = []
        edge_index = []
        edge_attr = []
        # Extract node features for subject and object entities
        for entity_key in ['subj', 'obj']:
            entity_info = (item[f'{entity_key}_type'], tuple(item['tokens'][item[f'{entity_key}_start']:item[f'{entity_key}_end']+1]))
            if entity_info not in entity_to_id:
                pos_one_hot = torch.tensor(item['stanford_pos_one_hot'][item[f'{entity_key}_start']], dtype=torch.float)
                ner_one_hot = torch.tensor(item['stanford_ner_one_hot'][item[f'{entity_key}_start']], dtype=torch.float)
                node_feature = torch.cat((pos_one_hot, ner_one_hot), dim=0)
                node_features.append(node_feature)
                entity_to_id[entity_info] = len(node_features) - 1
        # Find the shortest dependency path between subject and object
        dependency_path = get_shortest_dependency_path(item['stanford_head'], item['subj_start'], item['obj_start'])
        # Add node features for tokens in the dependency path
        for token_idx in dependency_path:
            if token_idx not in entity_to_id:
                pos_one_hot = torch.tensor(item['stanford_pos_one_hot'][token_idx], dtype=torch.float)
                ner_one_hot = torch.tensor(item['stanford_ner_one_hot'][token_idx], dtype=torch.float)
                node_feature = torch.cat((pos_one_hot, ner_one_hot), dim=0)
                node_features.append(node_feature)
                entity_to_id[token_idx] = len(node_features) - 1
        # Get IDs for subject and object entities
        subj_id = entity_to_id[(item['subj_type'], tuple(item['tokens'][item['subj_start']:item['subj_end']+1]))]
        obj_id = entity_to_id[(item['obj_type'], tuple(item['tokens'][item['obj_start']:item['obj_end']+1]))]
        # Add edge between subject and object with relation attribute
        edge_index.append([subj_id, obj_id])
        edge_attr.append(relation_to_one_hot(item['relation']))
        # Convert lists to tensors
        node_features = torch.stack(node_features)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.stack(edge_attr)
        # Create graph data object
        graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)
        graphs.append(graph_data)
    return graphs

# Create graph data for the preprocessed train and dev data
graphs_train = create_graphs(preprocessed_train_data)
graphs_dev = create_graphs(preprocessed_dev_data)

### 5. Constructing Graph Pairs for Contrastive Learning

We implement methods for forming balanced graph pairs for training a contrastive learning model. We extract relation labels from graph edge attributes and generate sets of positive (similar relation) and negative (dissimilar relation) graph pairs. This approach prepares the model for effective relation extraction using GNNs.

In [ ]:
def extract_relation_label(edge_attr):
    """
    Extract the relation label from the edge attributes.

    Args:
        edge_attr (torch.Tensor): Edge attributes.

    Returns:
        int: Index of the maximum value in the tensor, representing the relation label.
    """
    return edge_attr.argmax().item()

def form_balanced_graph_pairs(graphs, positive_per_graph=2, negative_per_graph=2):
    """
    Form balanced pairs of graphs for training.

    Args:
        graphs (list): List of graph data objects.
        positive_per_graph (int): Number of positive pairs per graph.
        negative_per_graph (int): Number of negative pairs per graph.

    Returns:
        list: List of balanced pairs of graphs.
    """
    # Organize graphs based on their relation labels
    relation_to_graphs = {}
    for graph in graphs:
        label = extract_relation_label(graph.edge_attr)
        if label not in relation_to_graphs:
            relation_to_graphs[label] = []
        relation_to_graphs[label].append(graph)

    # Initialize lists for positive and negative pairs
    positive_pairs, negative_pairs = [], []

    # Extract relation labels
    relations = list(relation_to_graphs.keys())

    # Generate positive pairs
    for relation, graph_list in relation_to_graphs.items():
        for graph in graph_list:
            possible_positives = [g for g in graph_list if g != graph]
            for _ in range(positive_per_graph):
                if possible_positives:
                    positive_partner = random.choice(possible_positives)
                    positive_pairs.append((graph, positive_partner, 1))

    # Generate negative pairs
    for relation, graph_list in relation_to_graphs.items():
        for graph in graph_list:
            other_relations = [r for r in relations if r != relation]
            for _ in range(negative_per_graph):
                if other_relations:
                    other_relation = random.choice(other_relations)
                    other_graphs = relation_to_graphs[other_relation]
                    if other_graphs:
                        negative_partner = random.choice(other_graphs)
                        negative_pairs.append((graph, negative_partner, 0))

    # Combine positive and negative pairs
    pairs = positive_pairs + negative_pairs
    random.shuffle(pairs)  # Shuffle pairs to mix positive and negative
    return pairs

In [ ]:
balanced_pairs = form_balanced_graph_pairs(graphs_train, positive_per_graph=2, negative_per_graph=2)

# Print information about the first 5 pairs
for pair in balanced_pairs[:5]:
    graph1, graph2, label = pair
    # Display label and number of nodes in each graph
    print(f"Pair label: {label}, Graph1 Nodes: {graph1.x.shape[0]}, Graph2 Nodes: {graph2.x.shape[0]}")

Pair label: 1, Graph1 Nodes: 8, Graph2 Nodes: 8
Pair label: 0, Graph1 Nodes: 5, Graph2 Nodes: 7
Pair label: 0, Graph1 Nodes: 7, Graph2 Nodes: 5
Pair label: 1, Graph1 Nodes: 10, Graph2 Nodes: 14
Pair label: 0, Graph1 Nodes: 7, Graph2 Nodes: 5


### 6. Graph Neural Network Model for Relation Extraction with Contrastive Learning

Our approach of implementing GNN model specifically designed for relation extraction, enhanced by contrastive learning consists of a method to compute the contrastive loss between pairs of graph embeddings which discerns between similar and dissimilar relations. The model architecture includes graph attention layers for feature extraction from nodes, followed by global mean pooling and fully connected layers for embedding generation and relation classification.

In [ ]:
def contrastive_loss(embedding1, embedding2, labels, margin=1.0):
    """
    Compute contrastive loss between pairs of embeddings.

    Args:
        embedding1 (torch.Tensor): Embeddings of the first item in the pair.
        embedding2 (torch.Tensor): Embeddings of the second item in the pair.
        labels (torch.Tensor): Labels indicating whether each pair is positive (1) or negative (0).
        margin (float): Margin for negative pairs.

    Returns:
        torch.Tensor: Contrastive loss.
    """
    loss = 0.0
    batch_size = embedding1.size(0)
    for i in range(batch_size):
        emb1 = embedding1[i].unsqueeze(0)
        emb2 = embedding2[i].unsqueeze(0)
        label = labels[i]
        if label == 1:
            # For positive pairs, compute Euclidean distance between embeddings
            loss += F.pairwise_distance(emb1, emb2).pow(2)
        else:
            # For negative pairs, enforce margin by computing hinge loss
            loss += F.relu(margin - F.pairwise_distance(emb1, emb2)).pow(2)
    # Average the loss over the batch
    return loss / batch_size

In [ ]:
class RelationExtractionGNN(torch.nn.Module):
    """
    Relation Extraction Graph Neural Network model.

    Args:
        num_node_features (int): Number of node features.
        num_classes (int): Number of relation classes.

    Attributes:
        conv1 (GATConv): Graph attention layer 1.
        conv2 (GATConv): Graph attention layer 2.
        fc1 (torch.nn.Linear): Fully connected layer 1.
        fc_relation_pred (torch.nn.Linear): Fully connected layer for relation prediction.
    """
    def __init__(self, num_node_features, num_classes):
        super(RelationExtractionGNN, self).__init__()
        # Define layers
        self.conv1 = GATConv(num_node_features, 128)
        self.conv2 = GATConv(128, 64)
        self.fc1 = torch.nn.Linear(64, 32)
        self.fc_relation_pred = torch.nn.Linear(32, num_classes)

    def forward(self, x, edge_index, batch):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Node features.
            edge_index (torch.Tensor): Graph edge indices.
            batch (torch.Tensor): Batch indices.

        Returns:
            tuple: Tuple containing embeddings and relation predictions.
        """
        # Graph convolution layers with ReLU activation
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        # Global pooling operation
        x = global_mean_pool(x, batch)
        # Fully connected layer with ReLU activation
        embedding = F.relu(self.fc1(x))
        # Final fully connected layer for relation prediction
        relation_pred = self.fc_relation_pred(embedding)
        return embedding, relation_pred

# Instantiate the RelationExtractionGNN model
model = RelationExtractionGNN(num_node_features=58, num_classes=42)

### 7. Training and Saving the GNN Model

Here we detail the training procedure for our model. We create a DataLoader for graph pairs, set up the loss function, and configure the optimizer for the training process. The model undergoes training over several epochs, during which it learns to optimize both the contrastive loss (to distinguish between similar and dissimilar graph pairs) and the relation classification loss. After training, we save the trained model.

In [ ]:
# Create DataLoader for balanced pairs of graphs
pair_loader = DataLoader(balanced_pairs, batch_size=32, shuffle=True)
# Define cross-entropy loss criterion
criterion = CrossEntropyLoss()
# Define Adam optimizer for model parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Margin for contrastive loss
margin = 2.0

# Iterate over epochs
for epoch in range(1, 3):
    total_loss = 0
    total_loss_epoch = 0
    # Iterate over data batches
    for data in pair_loader:
        graph1, graph2, labels = data
        optimizer.zero_grad()
        # Forward pass for the first graph
        embedding1, output1 = model(graph1.x, graph1.edge_index, graph1.batch)
        # Forward pass for the second graph
        embedding2, output2 = model(graph2.x, graph2.edge_index, graph2.batch)
        # Compute contrastive loss between embeddings
        c_loss = contrastive_loss(embedding1, embedding2, labels, margin=2.0)
        # Extract relation labels from edge attributes
        relation_labels1 = graph1.edge_attr.max(dim=1)[1]
        relation_labels2 = graph2.edge_attr.max(dim=1)[1]
        # Compute relation classification loss
        loss_relation1 = criterion(output1, relation_labels1)
        loss_relation2 = criterion(output2, relation_labels2)
        # Total loss is a combination of contrastive and relation classification losses
        total_loss = loss_relation1 + loss_relation2 + c_loss
        # Backpropagation and optimization step
        total_loss.backward()
        optimizer.step()
        total_loss_epoch += total_loss.item()
    # Print epoch and average loss over the data loader
    print(f'Epoch: {epoch}, Loss: {total_loss_epoch / len(pair_loader)}')

/Users/srikanthkuppili/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 1, Loss: 6.295862104071945
Epoch: 2, Loss: 4.866859180028321


In [ ]:
# Define the path where the trained model will be saved
model_path = 'my_trained_model.pth'

# Save the model's state dictionary to the specified path
torch.save({
    'model_state_dict': model.state_dict()  # Saving the state dictionary containing model parameters
}, model_path)

### 8. Evaluating the GNN Model on Test Data

Next, we evaluate the performance of our model on unseen test data. We load the pre-trained model, preprocess the test dataset, create graph data from it, and assemble a DataLoader for evaluation. The evaluation process calculates the model's accuracy, precision, recall, and F1 score over the test dataset.

In [ ]:
# Define the model architecture with the same parameters as used during training
loaded_model = RelationExtractionGNN(num_node_features=58, num_classes=42)

# Load the saved model parameters (state_dict) from the specified file
checkpoint = torch.load(model_path)

# Update the model's parameters with the loaded state_dict
loaded_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
# Preprocess the test dataset using the pre-defined tag indices for POS and NER tags
preprocessed_test_data = preprocess_dataset(test_data, pos_tag_to_index, ner_tag_to_index)

# Create graphs from the preprocessed test data
graphs_test = create_graphs(preprocessed_test_data)

# Create a DataLoader for the test graphs with a batch size of 32 and no shuffling
test_loader = DataLoader(graphs_test, batch_size=32, shuffle=False)

In [ ]:
def evaluate_model(model, loader):
    """
    Evaluate the performance of the model on the given data loader.

    Args:
    - model: The model to be evaluated
    - loader: DataLoader containing the dataset for evaluation

    Returns:
    - true_labels: List of true relation labels
    - predicted_labels: List of predicted relation labels
    - accuracy: Accuracy of the model on the evaluation dataset
    - precision: Precision score of the model on the evaluation dataset
    - recall: Recall score of the model on the evaluation dataset
    - f1: F1 score of the model on the evaluation dataset
    """
    model.eval()  # Set the model to evaluation mode
    true_labels = []  # List to store true relation labels
    predicted_labels = []  # List to store predicted relation labels

    with torch.no_grad():  # Turn off gradient tracking during evaluation
        for data in loader:  # Iterate through the data loader
            # Forward pass through the model to get embeddings and predictions
            embedding, output = model(data.x, data.edge_index, data.batch)

            # Predicted labels are obtained by selecting the class with maximum probability
            preds = output.argmax(dim=1)

            # True relation labels are decoded from one-hot encoding
            true_relations = data.edge_attr.argmax(dim=1)

            # Extend the lists with true and predicted labels
            true_labels.extend(true_relations.tolist())
            predicted_labels.extend(preds.tolist())

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

    return true_labels, predicted_labels, accuracy, precision, recall, f1

In [ ]:
# Evaluate the loaded model using the test data loader
true_labels, predicted_labels, accuracy, precision, recall, f1 = evaluate_model(loaded_model, test_loader)

# Print the evaluation metrics: test accuracy, precision, recall, and F1 score
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.7856
Precision: 0.6172
Recall: 0.7856
F1 Score: 0.6913


/Users/srikanthkuppili/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Scoring against Gold Standard TACRED Labels

We save the predicted relation labels into a text file for comparison with gold standard annotations file. We use an external scoring script (`score.py`) provided with TACRED to evaluate the model's predictions against a set of ground truth labels (`test.gold`).

In [ ]:
index_to_relation = {value: key for key, value in relation_to_index.items()}
predicted_labels_forEval = [index_to_relation[label_id] for label_id in predicted_labels]
true_labels_forEval = [index_to_relation[label_id] for label_id in true_labels]

In [ ]:
# Count the occurrences of unique pairs of true and predicted labels
truePred_count = {}  # Dictionary to store the count of each unique pair

# Iterate through true_labels and predicted_labels using zip
for comb in zip(true_labels_forEval,predicted_labels_forEval):
    truePred_count[comb] = truePred_count.get(comb, 0) + 1  # Increment the count for each unique pair or initialize to 1 if not present

truePred_count  # Return the dictionary containing the count of each unique pair

{('no_relation', 'no_relation'): 12184,
 ('per:title', 'no_relation'): 500,
 ('org:top_members/employees', 'no_relation'): 346,
 ('org:country_of_headquarters', 'no_relation'): 108,
 ('per:parents', 'no_relation'): 88,
 ('per:age', 'no_relation'): 200,
 ('per:countries_of_residence', 'no_relation'): 148,
 ('per:children', 'no_relation'): 37,
 ('org:alternate_names', 'no_relation'): 213,
 ('per:charges', 'no_relation'): 103,
 ('per:cities_of_residence', 'no_relation'): 189,
 ('per:origin', 'no_relation'): 132,
 ('org:founded_by', 'no_relation'): 68,
 ('per:employee_of', 'no_relation'): 264,
 ('per:siblings', 'no_relation'): 55,
 ('per:alternate_names', 'no_relation'): 11,
 ('org:website', 'no_relation'): 26,
 ('per:religion', 'no_relation'): 47,
 ('per:stateorprovince_of_death', 'no_relation'): 14,
 ('org:parents', 'no_relation'): 62,
 ('org:subsidiaries', 'no_relation'): 44,
 ('per:other_family', 'no_relation'): 60,
 ('per:stateorprovinces_of_residence', 'no_relation'): 81,
 ('org:memb

In [ ]:
# Open a file in write mode to store predicted labels for evaluation
with open('predicted_labels_forEval.txt', 'w') as file:
    # Write each predicted label to the file, with each label on a separate line
    for label in predicted_labels_forEval:
        file.write(f"{label}\n")

In [ ]:
# Run the "score.py" script with the test gold labels file and the predicted labels file as arguments
!python score.py test.gold predicted_labels_forEval.txt

Per-relation statistics:
org:alternate_names                  P: 100.00%  R:   0.00%  F1:   0.00%  #: 213
org:city_of_headquarters             P: 100.00%  R:   0.00%  F1:   0.00%  #: 82
org:country_of_headquarters          P: 100.00%  R:   0.00%  F1:   0.00%  #: 108
org:dissolved                        P: 100.00%  R:   0.00%  F1:   0.00%  #: 2
org:founded                          P: 100.00%  R:   0.00%  F1:   0.00%  #: 37
org:founded_by                       P: 100.00%  R:   0.00%  F1:   0.00%  #: 68
org:member_of                        P: 100.00%  R:   0.00%  F1:   0.00%  #: 18
org:members                          P: 100.00%  R:   0.00%  F1:   0.00%  #: 31
org:number_of_employees/members      P: 100.00%  R:   0.00%  F1:   0.00%  #: 19
org:parents                          P: 100.00%  R:   0.00%  F1:   0.00%  #: 62
org:political/religious_affiliation  P: 100.00%  R:   0.00%  F1:   0.00%  #: 10
org:shareholders                     P: 100.00%  R:   0.00%  F1:   0.00%  #: 13
org:stateorpro